# Detecting Payment Card Fraud

In this section, we'll look at a credit card fraud detection dataset, and build a binary classification model that can identify transactions as either fraudulent or valid, based on provided, *historical* data. In a [2016 study](https://nilsonreport.com/upload/content_promo/The_Nilson_Report_10-17-2016.pdf), it was estimated that credit card fraud was responsible for over 20 billion dollars in loss, worldwide. Accurately detecting cases of fraud is an ongoing area of research.

<img src=notebook_ims/fraud_detection.png width=50% />

### Labeled Data

The payment fraud data set (Dal Pozzolo et al. 2015) was downloaded from [Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud/data). This has features and labels for thousands of credit card transactions, each of which is labeled as fraudulent or valid. In this notebook, we'd like to train a model based on the features of these transactions so that we can predict risky or fraudulent transactions in the future.

### Binary Classification

Since we have true labels to aim for, we'll take a **supervised learning** approach and train a binary classifier to sort data into one of our two transaction classes: fraudulent or valid.  We'll train a model on training data and see how well it generalizes on some test data.

The notebook will be broken down into a few steps:
* Loading and exploring the data
* Splitting the data into train/test sets
* Defining and training a LinearLearner, binary classifier
* Making improvements on the model
* Evaluating and comparing model test performance

### Making Improvements

A lot of this notebook will focus on making improvements, as discussed in [this SageMaker blog post](https://aws.amazon.com/blogs/machine-learning/train-faster-more-flexible-models-with-amazon-sagemaker-linear-learner/). Specifically, we'll address techniques for:

1. **Tuning a model's hyperparameters** and aiming for a specific metric, such as high recall or precision.
2. **Managing class imbalance**, which is when we have many more training examples in one class than another (in this case, many more valid transactions than fraudulent).

---

First, import the usual resources.

In [1]:
import io
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import boto3
import sagemaker
from sagemaker import get_execution_role

%matplotlib inline
%config inlinebackend.figure_format = 'retina'

I'm storing my **SageMaker variables** in the next cell:
* sagemaker_session: The SageMaker session we'll use for training models.
* bucket: The name of the default S3 bucket that we'll use for data storage.
* role: The IAM role that defines our data and model permissions.

In [2]:
# sagemaker session, role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# s3 bucket name
bucket = sagemaker_session.default_bucket()


## Loading and Exploring the Data

Next, I am loading the data and unzipping the data in the file `creditcardfraud.zip`. This directory will hold one csv file of all the transaction data, `creditcard.csv`.

As in previous notebooks, it's important to look at the distribution of data since this will inform how we develop a fraud detection model. We'll want to know: How many data points we have to work with, the number and type of features, and finally, the distribution of data over the classes (valid or fraudulent).

In [4]:
# only have to run once
!wget https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c534768_creditcardfraud/creditcardfraud.zip
!unzip creditcardfraud


--2021-07-14 18:48:05--  https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c534768_creditcardfraud/creditcardfraud.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.192.24
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.192.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69155632 (66M) [application/zip]
Saving to: ‘creditcardfraud.zip’

creditcardfraud.zip 100%[===================>]  65.95M  10.6MB/s    in 6.7s    

2021-07-14 18:48:12 (9.87 MB/s) - ‘creditcardfraud.zip’ saved [69155632/69155632]

Archive:  creditcardfraud.zip
  inflating: creditcard.csv          


In [3]:
# read in the csv file
local_data = 'creditcard.csv'

# print out some data
transaction_df = pd.read_csv(local_data)
print('Data shape (rows, cols): ', transaction_df.shape)
print()
transaction_df.head()

Data shape (rows, cols):  (284807, 31)



,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
transaction_df.value_counts('Class')

Class
0    284315
1       492
dtype: int64

### EXERCISE: Calculate the percentage of fraudulent data

Take a look at the distribution of this transaction data over the classes, valid and fraudulent. 

Complete the function `fraudulent_percentage`, below. Count up the number of data points in each class and calculate the *percentage* of the data points that are fraudulent.

In [5]:
# Calculate the fraction of data points that are fraudulent
def fraudulent_percentage(transaction_df):
    '''Calculate the fraction of all data points that have a 'Class' label of 1; fraudulent.
       :param transaction_df: Dataframe of all transaction data points; has a column 'Class'
       :return: A fractional percentage of fraudulent data points/all points
    '''
    
    counts = transaction_df.value_counts('Class')
    total = counts[0] + counts[1]
    fraud = counts[1]
    
    return fraud / total

Test out your code by calling your function and printing the result.

In [6]:
# call the function to calculate the fraud percentage
fraud_percentage = fraudulent_percentage(transaction_df)

print('Fraudulent percentage = ', fraud_percentage*100)
print('Total # of fraudulent pts: ', fraud_percentage*transaction_df.shape[0])
print('Out of (total) pts: ', transaction_df.shape[0])


Fraudulent percentage =  0.1727485630620034
Total # of fraudulent pts:  492.0
Out of (total) pts:  284807


In [7]:
transaction_df.shape[0]

284807

In [8]:
transaction_df.iloc[:,-1].head()

0    0
1    0
2    0
3    0
4    0
Name: Class, dtype: int64

### EXERCISE: Split into train/test datasets

In this example, we'll want to evaluate the performance of a fraud classifier; training it on some training data and testing it on *test data* that it did not see during the training process. So, we'll need to split the data into separate training and test sets.

Complete the `train_test_split` function, below. This function should:
* Shuffle the transaction data, randomly
* Split it into two sets according to the parameter `train_frac`
* Get train/test features and labels
* Return the tuples: (train_features, train_labels), (test_features, test_labels)

In [9]:
# split into train/test
def split_data(transaction_df, test_size= 0.3, random_state=1):
    '''Shuffle the data and randomly split into train and test sets;
       separate the class labels (the column in transaction_df) from the features.
       
       Parameters 
            df: Dataframe of all credit card transaction data
            train_frac: The decimal fraction of data that should be training data
            seed: Random seed for shuffling and reproducibility, default = 1
       Return
            train_features, train_labels test_features, test_labels
    '''
    X = transaction_df.iloc[:,:-1]
    y = transaction_df.iloc[:,-1]
    
    # shuffle and split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    train_features = X_train
    train_labels = y_train
    test_features = X_test
    test_labels = y_test
    
    return train_features, test_features, train_labels, test_labels


### Test Cell

In the cells below, I'm creating the train/test data and checking to see that result makes sense. The tests below test that the above function splits the data into the expected number of points and that the labels are indeed, class labels (0, 1).

In [10]:
# get train/test data
train_features, test_features, train_labels, test_labels = split_data(transaction_df, test_size=0.3, random_state=23)

In [11]:
# manual test

# for a split of 0.7:0.3 there should be ~2.33x as many training as test pts
print('Training data pts: ', len(train_features))
print('Test data pts: ', len(test_features))
print()

# take a look at first item and see that it aligns with first row of data
print('First item: \n', train_features.iloc[:,0])
print('Label: ', train_labels[0])
print()

# test split
assert len(train_features) > 2.333*len(test_features), \
        'Unexpected number of train/test points for a train_frac=0.7'
# test labels
assert np.all(train_labels)== 0 or np.all(train_labels)== 1, \
        'Train labels should be 0s or 1s.'
assert np.all(test_labels)== 0 or np.all(test_labels)== 1, \
        'Test labels should be 0s or 1s.'
print('Tests passed!')

Training data pts:  199364
Test data pts:  85443

First item: 
 162807    115388.0
261464    160030.0
87743      61817.0
221699    142664.0
191148    129135.0
            ...   
202783    134489.0
271848    164780.0
223177    143276.0
271400    164558.0
127718     78440.0
Name: Time, Length: 199364, dtype: float64
Label:  0

Tests passed!


---
# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

In this notebook, you'll define and train the SageMaker, built-in algorithm, [LinearLearner](https://sagemaker.readthedocs.io/en/stable/linear_learner.html). 

A LinearLearner has two main applications:
1. For regression tasks in which a linear line is fit to some data points, and you want to produce a predicted output value given some data point (example: predicting house prices given square area).
2. For binary classification, in which a line is separating two classes of data and effectively outputs labels; either 1 for data that falls above the line or 0 for points that fall on or below the line.

<img src='notebook_ims/linear_separator.png' width=50% />

In this case, we'll be using it for case 2, and we'll train it to separate data into our two classes: valid or fraudulent. 

### EXERCISE: Create a LinearLearner Estimator

You've had some practice instantiating built-in models in SageMaker. All estimators require some constructor arguments to be passed in. See if you can complete this task, instantiating a LinearLearner estimator, using only the [LinearLearner documentation](https://sagemaker.readthedocs.io/en/stable/linear_learner.html) as a resource. This takes in a lot of arguments, but not all are required. My suggestion is to start with a simple model, utilizing default values where applicable. Later, we will discuss some specific hyperparameters and their use cases.

#### Instance Types

It is suggested that you use instances that are available in the free tier of usage: `'ml.c4.xlarge'` for training and `'ml.t2.medium'` for deployment.

In [12]:
# import LinearLearner
from sagemaker import LinearLearner

prefix = 'creditcard'
output_path = 's3://{}/{}/'.format(bucket, prefix)
print('Training artifacts will be uploaded to ', output_path)

Training artifacts will be uploaded to  s3://sagemaker-us-west-2-904606187431/creditcard/


In [13]:
linear_learner = LinearLearner(role=role,
                               instance_count=1,
                               instance_type='ml.c4.xlarge',
                               output_path=output_path,
                               sagemaker_session=sagemaker_session,
                               predictor_type='binary_classifier',
                               epochs=16)

### EXERCISE: Convert data into a RecordSet format

Next, prepare the data for a built-in model by converting the train features and labels into numpy array's of float values. Then you can use the [record_set function](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearner.record_set) to format the data as a RecordSet and prepare it for training!

In [14]:
# create RecordSet of training data
X_train_np = train_features.values.astype('float32')
y_train_np = train_labels.values.astype('float32')

formatted_train_data = linear_learner.record_set(X_train_np, labels=y_train_np)

### EXERCISE: Train the Estimator

After instantiating your estimator, train it with a call to `.fit()`, passing in the formatted training data.

In [45]:
%%time 
# train the estimator on formatted training data

linear_learner.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-07-14 20:16:06 Starting - Starting the training job...
2021-07-14 20:16:30 Starting - Launching requested ML instancesProfilerReport-1626293766: InProgress
...
2021-07-14 20:17:01 Starting - Preparing the instances for training.........
2021-07-14 20:18:30 Downloading - Downloading input data...
2021-07-14 20:18:54 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script
[07/14/2021 20:19:20 INFO 140283928004416] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'a

[2021-07-14 20:19:34.657] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 7212, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626293974.6577022, "EndTime": 1626293974.6577873, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.01692018417377568, "count": 1, "min": 0.01692018417377568, "max": 0.01692018417377568}}}

#metrics {"StartTime": 1626293974.6578739, "EndTime": 1626293974.6578948, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.014929442889726342, "count": 1, "min": 0.014929442889726342, "max": 0.014929442889726342}}}

#metrics {"StartTime": 1626293974.6579537, "EndTime": 1626293974.6579719, "Dimensions": {"Algorithm": "Linear L

[2021-07-14 20:19:48.118] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 6646, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626293988.1186197, "EndTime": 1626293988.118712, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.00804507978238053, "count": 1, "min": 0.00804507978238053, "max": 0.00804507978238053}}}

#metrics {"StartTime": 1626293988.1187973, "EndTime": 1626293988.118816, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.006706084989423129, "count": 1, "min": 0.006706084989423129, "max": 0.006706084989423129}}}

#metrics {"StartTime": 1626293988.118874, "EndTime": 1626293988.118892, "Dimensions": {"Algorithm": "Linear Learn

[2021-07-14 20:19:54.459] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 6327, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626293994.4596355, "EndTime": 1626293994.459719, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0069460316758659015, "count": 1, "min": 0.0069460316758659015, "max": 0.0069460316758659015}}}

#metrics {"StartTime": 1626293994.4598048, "EndTime": 1626293994.4598207, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.00576958492772663, "count": 1, "min": 0.00576958492772663, "max": 0.00576958492772663}}}

#metrics {"StartTime": 1626293994.4598742, "EndTime": 1626293994.4598866, "Dimensions": {"Algorithm": "Linea

...

[2021-07-14 20:20:35.232] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 23, "duration": 6977, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626294035.2329814, "EndTime": 1626294035.2330563, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005199369048952457, "count": 1, "min": 0.005199369048952457, "max": 0.005199369048952457}}}

#metrics {"StartTime": 1626294035.2331505, "EndTime": 1626294035.2331674, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004471952374257035, "count": 1, "min": 0.004471952374257035, "max": 0.004471952374257035}}}

#metrics {"StartTime": 1626294035.2332199, "EndTime": 1626294035.2332351, "Dimensions": {"Algorithm": "Li

[2021-07-14 20:20:48.815] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 6847, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626294048.8154047, "EndTime": 1626294048.8154917, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0050442263522938865, "count": 1, "min": 0.0050442263522938865, "max": 0.0050442263522938865}}}

#metrics {"StartTime": 1626294048.8155873, "EndTime": 1626294048.8156059, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004386274271574452, "count": 1, "min": 0.004386274271574452, "max": 0.004386274271574452}}}

#metrics {"StartTime": 1626294048.8156672, "EndTime": 1626294048.815686, "Dimensions": {"Algorithm": "

[2021-07-14 20:20:55.270] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 6442, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626294055.2710197, "EndTime": 1626294055.2711017, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0049885974972691365, "count": 1, "min": 0.0049885974972691365, "max": 0.0049885974972691365}}}

#metrics {"StartTime": 1626294055.27118, "EndTime": 1626294055.2711997, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004357680515428284, "count": 1, "min": 0.004357680515428284, "max": 0.004357680515428284}}}

#metrics {"StartTime": 1626294055.2712657, "EndTime": 1626294055.2712827, "Dimensions": {"Algorithm": "L

[2021-07-14 20:21:09.250] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 33, "duration": 7124, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626294069.2503245, "EndTime": 1626294069.2504041, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 15, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004905172780530536, "count": 1, "min": 0.004905172780530536, "max": 0.004905172780530536}}}

#metrics {"StartTime": 1626294069.2504883, "EndTime": 1626294069.2505038, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 15, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004315181595895758, "count": 1, "min": 0.004315181595895758, "max": 0.004315181595895758}}}

#metrics {"StartTime": 1626294069.2505455, "EndTime": 1626294069.2505562, "Dimensions": {"Algorithm": "Li


2021-07-14 20:21:31 Uploading - Uploading generated training model
2021-07-14 20:21:31 Completed - Training job completed
ProfilerReport-1626293766: NoIssuesFound
Training seconds: 179
Billable seconds: 179
CPU times: user 819 ms, sys: 46.8 ms, total: 865 ms
Wall time: 5min 43s


### EXERCISE: Deploy the trained model

Deploy your model to create a predictor. We'll use this to make predictions on our test data and evaluate the model.

In [46]:
%%time 
# deploy and create a predictor
linear_predictor = linear_learner.deploy(initial_instance_count=1,
                                         instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-----------------!CPU times: user 297 ms, sys: 5.58 ms, total: 303 ms
Wall time: 8min 32s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to the test data.

According to the deployed [predictor documentation](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearnerPredictor), this predictor expects an `ndarray` of input features and returns a list of Records.
> "The prediction is stored in the "predicted_label" key of the `Record.label` field."

Let's first test our model on just one test point, to see the resulting list.

In [51]:
test_features.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
234613,148047.0,2.028685,0.224580,-2.551702,0.402302,0.905105,-1.076018,0.654887,-0.331324,-0.156304,...,-0.123836,0.090078,0.352363,-0.043347,0.736527,0.351725,0.655745,-0.111035,-0.055465,25.43
100032,67391.0,1.255790,-0.830174,0.559989,-0.395207,-1.312711,-0.744008,-0.595538,-0.112043,-0.295092,...,0.144910,-0.054741,-0.259250,-0.019760,0.397157,0.396381,-0.338241,0.004523,0.027822,75.95
171934,120883.0,2.268610,-1.579083,-0.434778,-1.621200,-1.567335,-0.091604,-1.672820,0.060484,-0.950334,...,-0.437520,0.061080,0.772053,0.102445,-0.483384,-0.176745,0.022972,0.042990,-0.059089,4.05
240497,150625.0,2.021563,-0.412214,-2.559774,-0.717091,2.373563,3.323449,-0.459101,0.787639,0.401595,...,-0.124227,-0.284213,-0.842771,0.368431,0.698954,-0.270343,0.217665,-0.054984,-0.064311,15.99
125730,77777.0,1.194713,0.285594,0.172347,0.445008,0.079844,-0.077947,-0.066638,0.073723,-0.286556,...,-0.058691,-0.224247,-0.621006,0.100041,-0.334259,0.174678,0.125351,-0.007367,0.014924,1.29


In [74]:
# test one prediction
test_x_np = test_features.values.astype('float32')
result = linear_predictor.predict(test_x_np[0])

print(result)
print()
print("Predicted label -> ", result[0].label['predicted_label'].float32_tensor.values[0])

[label {
  key: "predicted_label"
  value {
    float32_tensor {
      values: 0.0
    }
  }
}
label {
  key: "score"
  value {
    float32_tensor {
      values: 0.00101965491194278
    }
  }
}
]

Predicted label ->  0.0


### Helper function for evaluation


The provided function below, takes in a deployed predictor, some test features and labels, and returns a dictionary of metrics; calculating false negatives and positives as well as recall, precision, and accuracy.

In [15]:
# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
        Evaluate a model on a test set given the prediction endpoint.  
        Return binary classification metrics.
        
        Params
            predictor: A prediction endpoint
            test_features: Test features
            test_labels: Class labels for test data
            verbose: If True, prints a table of all performance metrics
        Return
            A dictionary of performance metrics.
    """
    
    # We have a lot of test data, so we'll split it into batches of 100
    # split the test data set into batches and evaluate using prediction endpoint    
    prediction_batches = [predictor.predict(batch) for batch in np.array_split(test_features, 100)]
    
    # LinearLearner produces a `predicted_label` for each data point in a batch
    # get the 'predicted_label' for every point in a batch
    test_preds = np.concatenate([np.array([x.label['predicted_label'].float32_tensor.values[0] for x in batch]) 
                                 for batch in prediction_batches])
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # printing a table of metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}


### Test Results

The cell below runs the `evaluate` function. 

The code assumes that you have a defined `predictor` and `test_features` and `test_labels` from previously-run cells.

In [75]:
print('Metrics for simple, LinearLearner.\n')

# get metrics for linear predictor
metrics = evaluate(linear_predictor, 
                   test_features.values.astype('float32'), 
                   test_labels, 
                   verbose=True) # verbose means we'll print out the metrics


Metrics for simple, LinearLearner.

prediction (col)    0.0  1.0
actual (row)                
0                 85265   39
1                    21  118

Recall:     0.849
Precision:  0.752
Accuracy:   0.999



## Delete the Endpoint

I've added a convenience function to delete prediction endpoints after we're done with them. And if you're done evaluating the model, you should delete your model endpoint!

In [76]:
# Deletes a precictor.endpoint
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [77]:
# delete the predictor endpoint 
delete_endpoint(linear_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-07-14-20-22-16-558


---

# Model Improvements

The default LinearLearner got a high accuracy, but still classified fraudulent and valid data points incorrectly. Specifically classifying more than 30 points as false negatives (incorrectly labeled, fraudulent transactions), and a little over 30 points as false positives (incorrectly labeled, valid transactions). Let's think about what, during training, could cause this behavior and what we could improve.

**1. Model optimization**
* If we imagine that we are designing this model for use in a bank application, we know that users do *not* want any valid transactions to be categorized as fraudulent. That is, we want to have as few **false positives** (0s classified as 1s) as possible. 
* On the other hand, if our bank manager asks for an application that will catch almost *all* cases of fraud, even if it means a higher number of false positives, then we'd want as few **false negatives** as possible.
* To train according to specific product demands and goals, we do not want to optimize for accuracy only. Instead, we want to optimize for a metric that can help us decrease the number of false positives or negatives. 

<img src='notebook_ims/precision_recall.png' width=40% />
     
In this notebook, we'll look at different cases for tuning a model and make an optimization decision, accordingly.

**2. Imbalanced training data**
* At the start of this notebook, we saw that only about 0.17% of the training data was labeled as fraudulent. So, even if a model labels **all** of our data as valid, it will still have a high accuracy. 
* This may result in some overfitting towards valid data, which accounts for some **false negatives**; cases in which fraudulent data (1) is incorrectly characterized as valid (0).

So, let's address these issues in order; first, tuning our model and optimizing for a specific metric during training, and second, accounting for class imbalance in the training set. 


## Improvement: Model Tuning

Optimizing according to a specific metric is called **model tuning**, and SageMaker provides a number of ways to automatically tune a model.


### Create a LinearLearner and tune for higher precision 

**Scenario:**
* A bank has asked you to build a model that detects cases of fraud with an accuracy of about 85%. 

In this case, we want to build a model that has as many true positives and as few false negatives, as possible. This corresponds to a model with a high **recall**: true positives / (true positives + false negatives). 

To aim for a specific metric, LinearLearner offers the hyperparameter `binary_classifier_model_selection_criteria`, which is the model evaluation criteria for the training dataset. A reference to this parameter is in [LinearLearner's documentation](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearner). We'll also have to further specify the exact value we want to aim for; read more about the details of the parameters, [here](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html).

I will assume that performance on a training set will be within about 5% of the performance on a test set. So, for a recall of about 85%, I'll aim for a bit higher, 90%.

In [17]:
# instantiate a LinearLearner
# tune the model for a higher recall
linear_recall = LinearLearner(role=role,
                              instance_count=1, 
                              instance_type='ml.c4.xlarge',
                              predictor_type='binary_classifier',
                              output_path=output_path,
                              sagemaker_session=sagemaker_session,
                              epochs=15,
                              binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                              target_recall=0.9) # 90% recall


### Train the tuned estimator

Fit the new, tuned estimator on the formatted training data.

In [18]:
%%time 
# train the estimator on formatted training data
linear_recall.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-07-15 21:26:09 Starting - Starting the training job...
2021-07-15 21:26:36 Starting - Launching requested ML instancesProfilerReport-1626384369: InProgress
......
2021-07-15 21:27:37 Starting - Preparing the instances for training............
2021-07-15 21:29:37 Downloading - Downloading input data...
2021-07-15 21:30:02 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[07/15/2021 21:30:09 INFO 140229055596352] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'ini

[2021-07-15 21:30:30.781] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 6915, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626384630.781896, "EndTime": 1626384630.7819762, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.010319409650773858, "count": 1, "min": 0.010319409650773858, "max": 0.010319409650773858}}}

#metrics {"StartTime": 1626384630.782048, "EndTime": 1626384630.782066, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.008745735532674359, "count": 1, "min": 0.008745735532674359, "max": 0.008745735532674359}}}

#metrics {"StartTime": 1626384630.7821198, "EndTime": 1626384630.782137, "Dimensions": {"Algorithm": "Linear Le

[2021-07-15 21:30:37.237] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 6443, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626384637.2372336, "EndTime": 1626384637.2374227, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.00804507978238053, "count": 1, "min": 0.00804507978238053, "max": 0.00804507978238053}}}

#metrics {"StartTime": 1626384637.237501, "EndTime": 1626384637.2375174, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.006706084989423129, "count": 1, "min": 0.006706084989423129, "max": 0.006706084989423129}}}

#metrics {"StartTime": 1626384637.2375667, "EndTime": 1626384637.2375777, "Dimensions": {"Algorithm": "Linear Le

[2021-07-15 21:30:50.745] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 13, "duration": 6726, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626384650.7457485, "EndTime": 1626384650.7458324, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 5, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.006317862431607654, "count": 1, "min": 0.006317862431607654, "max": 0.006317862431607654}}}

#metrics {"StartTime": 1626384650.7459152, "EndTime": 1626384650.7459338, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 5, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005260928830309729, "count": 1, "min": 0.005260928830309729, "max": 0.005260928830309729}}}

#metrics {"StartTime": 1626384650.7459955, "EndTime": 1626384650.7460136, "Dimensions": {"Algorithm": "Line

...

[2021-07-15 21:31:18.742] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 21, "duration": 6951, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626384678.7422523, "EndTime": 1626384678.742331, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 9, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005309705836689053, "count": 1, "min": 0.005309705836689053, "max": 0.005309705836689053}}}

#metrics {"StartTime": 1626384678.7424173, "EndTime": 1626384678.742438, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 9, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004539450900938043, "count": 1, "min": 0.004539450900938043, "max": 0.004539450900938043}}}

#metrics {"StartTime": 1626384678.742492, "EndTime": 1626384678.7425122, "Dimensions": {"Algorithm": "Linear 

[2021-07-15 21:31:32.570] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 25, "duration": 6953, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626384692.5706537, "EndTime": 1626384692.570731, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 11, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005113061845003061, "count": 1, "min": 0.005113061845003061, "max": 0.005113061845003061}}}

#metrics {"StartTime": 1626384692.5708134, "EndTime": 1626384692.5708282, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 11, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004423530006828021, "count": 1, "min": 0.004423530006828021, "max": 0.004423530006828021}}}

#metrics {"StartTime": 1626384692.570869, "EndTime": 1626384692.5708797, "Dimensions": {"Algorithm": "Line

[2021-07-15 21:31:39.379] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 6797, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626384699.3791063, "EndTime": 1626384699.3791924, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0050442263522938865, "count": 1, "min": 0.0050442263522938865, "max": 0.0050442263522938865}}}

#metrics {"StartTime": 1626384699.379281, "EndTime": 1626384699.3793008, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004386274271574452, "count": 1, "min": 0.004386274271574452, "max": 0.004386274271574452}}}

#metrics {"StartTime": 1626384699.379353, "EndTime": 1626384699.379371, "Dimensions": {"Algorithm": "Li

[2021-07-15 21:31:46.075] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 6685, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626384706.0757244, "EndTime": 1626384706.075806, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0049885974972691365, "count": 1, "min": 0.0049885974972691365, "max": 0.0049885974972691365}}}

#metrics {"StartTime": 1626384706.0758743, "EndTime": 1626384706.0758913, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004357680515428284, "count": 1, "min": 0.004357680515428284, "max": 0.004357680515428284}}}

#metrics {"StartTime": 1626384706.0759487, "EndTime": 1626384706.075965, "Dimensions": {"Algorithm": "L


2021-07-15 21:32:04 Uploading - Uploading generated training model
2021-07-15 21:32:04 Completed - Training job completed
Training seconds: 154
Billable seconds: 154
CPU times: user 901 ms, sys: 65.8 ms, total: 967 ms
Wall time: 6min 13s


### Deploy and evaluate the tuned estimator

Deploy the tuned predictor and evaluate it.

We hypothesized that a tuned model, optimized for a higher recall, would have fewer false negatives (fraudulent transactions incorrectly labeled as valid); did the number of false negatives get reduced after tuning the model?

In [19]:
%%time 
# deploy and create a predictor
recall_predictor = linear_recall.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-------------------!CPU times: user 333 ms, sys: 13.5 ms, total: 347 ms
Wall time: 9min 32s


In [21]:
print('Metrics for tuned (recall), LinearLearner.\n')

# get metrics for tuned predictor
metrics = evaluate(recall_predictor, 
                   test_features.values.astype('float32'), 
                   test_labels, 
                   verbose=True)

Metrics for tuned (recall), LinearLearner.

prediction (col)    0.0   1.0
actual (row)                 
0                 81105  4199
1                     4   135

Recall:     0.971
Precision:  0.031
Accuracy:   0.951



## Delete the endpoint 

As always, when you're done evaluating a model, you should delete the endpoint. Below, I'm using the `delete_endpoint` helper function I defined earlier.

In [22]:
# delete the predictor endpoint 
# delete_endpoint(recall_predictor)
recall_predictor.delete_endpoint() # new way to do it

---
## Improvement: Managing Class Imbalance

We have a model that is tuned to get a higher recall, which aims to reduce the number of false negatives. Earlier, we discussed how class imbalance may actually bias our model towards predicting that all transactions are valid, resulting in higher false negatives and true negatives. It stands to reason that this model could be further improved if we account for this imbalance.

To account for class imbalance during training of a binary classifier, LinearLearner offers the hyperparameter, `positive_example_weight_mult`, which is the weight assigned to positive (1, fraudulent) examples when training a binary classifier. The weight of negative examples (0, valid) is fixed at 1. 

### EXERCISE: Create a LinearLearner with a `positive_example_weight_mult` parameter

In **addition** to tuning a model for higher recall (you may use `linear_recall` as a starting point), you should *add* a parameter that helps account for class imbalance. From the [hyperparameter documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html) on `positive_example_weight_mult`, it reads:
> "If you want the algorithm to choose a weight so that errors in classifying negative vs. positive examples have equal impact on training loss, specify `balanced`."

You could also put in a specific float value, in which case you'd want to weight positive examples more heavily than negative examples, since there are fewer of them.

In [25]:
# instantiate a LinearLearner

# include params for tuning for higher recall
# *and* account for class imbalance in training data
linear_balanced = LinearLearner(role=role,
                                instance_count=1,
                                instance_type='ml.c4.xlarge',
                                output_path=output_path,
                                predictor_type='binary_classifier',
                                sagemaker_session=sagemaker_session,
                                epochs=16,
                                binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                                target_recall=0.9,
                                positive_example_weight_mult='balanced')


### EXERCISE: Train the balanced estimator

Fit the new, balanced estimator on the formatted training data.

In [26]:
%%time 
# train the estimator on formatted training data
linear_balanced.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-07-15 21:46:12 Starting - Starting the training job...
2021-07-15 21:46:35 Starting - Launching requested ML instancesProfilerReport-1626385572: InProgress
......
2021-07-15 21:47:35 Starting - Preparing the instances for training............
2021-07-15 21:49:36 Downloading - Downloading input data
2021-07-15 21:49:36 Training - Downloading the training image...
2021-07-15 21:49:56 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[07/15/2021 21:50:02 INFO 140106812909376] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method'

[2021-07-15 21:50:18.038] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 7549, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626385818.0388627, "EndTime": 1626385818.0389488, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.31023651310906336, "count": 1, "min": 0.31023651310906336, "max": 0.31023651310906336}}}

#metrics {"StartTime": 1626385818.039045, "EndTime": 1626385818.039067, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2847244280331099, "count": 1, "min": 0.2847244280331099, "max": 0.2847244280331099}}}

#metrics {"StartTime": 1626385818.03912, "EndTime": 1626385818.0391347, "Dimensions": {"Algorithm": "

[2021-07-15 21:50:31.696] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 6632, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626385831.696589, "EndTime": 1626385831.6966748, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2675552611231205, "count": 1, "min": 0.2675552611231205, "max": 0.2675552611231205}}}

#metrics {"StartTime": 1626385831.6967688, "EndTime": 1626385831.6967893, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.255248536421426, "count": 1, "min": 0.255248536421426, "max": 0.255248536421426}}}

#metrics {"StartTime": 1626385831.6968427, "EndTime": 1626385831.6968548, "Dimensions": {"Algorithm": "Lin

[2021-07-15 21:50:38.671] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 6962, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626385838.6714575, "EndTime": 1626385838.6715443, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2603724578397358, "count": 1, "min": 0.2603724578397358, "max": 0.2603724578397358}}}

#metrics {"StartTime": 1626385838.6716263, "EndTime": 1626385838.6716456, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.250463905813706, "count": 1, "min": 0.250463905813706, "max": 0.250463905813706}}}

#metrics {"StartTime": 1626385838.6716986, "EndTime": 1626385838.6717148, "Dimensions": {"Algorithm": "L

...

[2021-07-15 21:51:20.815] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 23, "duration": 6469, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626385880.8152614, "EndTime": 1626385880.815347, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.24626800211230715, "count": 1, "min": 0.24626800211230715, "max": 0.24626800211230715}}}

#metrics {"StartTime": 1626385880.8154194, "EndTime": 1626385880.8154366, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.24135710939570287, "count": 1, "min": 0.24135710939570287, "max": 0.24135710939570287}}}

#metrics {"StartTime": 1626385880.8154867, "EndTime": 1626385880.8155022, "Dimensions": {"Algo

[2021-07-15 21:51:34.438] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 6886, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626385894.4384084, "EndTime": 1626385894.4384847, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2447554219308211, "count": 1, "min": 0.2447554219308211, "max": 0.2447554219308211}}}

#metrics {"StartTime": 1626385894.438572, "EndTime": 1626385894.4385924, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2402962157858077, "count": 1, "min": 0.2402962157858077, "max": 0.2402962157858077}}}

#metrics {"StartTime": 1626385894.4386547, "EndTime": 1626385894.4386733, "Dimensions": {"Algorithm"

[2021-07-15 21:51:41.737] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 7286, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626385901.7370489, "EndTime": 1626385901.7371297, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.24421366560279426, "count": 1, "min": 0.24421366560279426, "max": 0.24421366560279426}}}

#metrics {"StartTime": 1626385901.7372172, "EndTime": 1626385901.7372372, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.23987274275353207, "count": 1, "min": 0.23987274275353207, "max": 0.23987274275353207}}}

#metrics {"StartTime": 1626385901.7373023, "EndTime": 1626385901.7373204, "Dimensions": {"Alg

[2021-07-15 21:51:54.829] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 33, "duration": 6423, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626385914.8294015, "EndTime": 1626385914.829584, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 15, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2434097470997566, "count": 1, "min": 0.2434097470997566, "max": 0.2434097470997566}}}

#metrics {"StartTime": 1626385914.829668, "EndTime": 1626385914.8296885, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 15, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2391293906973834, "count": 1, "min": 0.2391293906973834, "max": 0.2391293906973834}}}

#metrics {"StartTime": 1626385914.8297508, "EndTime": 1626385914.8297687, "Dimensions": {"Algorithm":


2021-07-15 21:52:08 Uploading - Uploading generated training model
2021-07-15 21:52:08 Completed - Training job completed
Training seconds: 165
Billable seconds: 165
CPU times: user 914 ms, sys: 52.6 ms, total: 967 ms
Wall time: 6min 14s


### EXERCISE: Deploy and evaluate the balanced estimator

Deploy the balanced predictor and evaluate it. Do the results match with your expectations?

In [29]:
%%time 
# deploy and create a predictor
balanced_predictor = linear_balanced.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-----------------!CPU times: user 309 ms, sys: 5.55 ms, total: 315 ms
Wall time: 8min 32s


In [31]:
print('Metrics for balanced, LinearLearner.\n')

# get metrics for balanced predictor
metrics = evaluate(balanced_predictor, 
                   test_features.values.astype('float32'), 
                   test_labels, 
                   verbose=True)

Metrics for balanced, LinearLearner.

prediction (col)    0.0   1.0
actual (row)                 
0                 83890  1414
1                     6   133

Recall:     0.957
Precision:  0.086
Accuracy:   0.983



## Delete the endpoint 

When you're done evaluating a model, you should delete the endpoint.

In [32]:
# delete the predictor endpoint 
balanced_predictor.delete_endpoint()

A note on metric variability: 

The above model is tuned for the best possible precision with recall fixed at about 90%. The recall is fixed at 90% during training, but may vary when we apply our trained model to a test set of data.

---
## Model Design

Now that you've seen how to tune and balance a LinearLearner. Create, train and deploy your own model. This exercise is meant to be more open-ended, so that you get practice with the steps involved in designing a model and deploying it.

### EXERCISE: Train and deploy a LinearLearner with appropriate hyperparameters, according to the given scenario

**Scenario:**
* A bank has asked you to build a model that optimizes for a good user experience; users should only ever have up to about 15% of their valid transactions flagged as fraudulent.

This requires that you make a design decision: Given the above scenario, what metric (and value) should you aim for during training?

You may assume that performance on a training set will be within about 5-10% of the performance on a test set. For example, if you get 80% on a training set, you can assume that you'll get between about 70-90% accuracy on a test set.

Your final model should account for class imbalance and be appropriately tuned. 

In [33]:
%%time
# instantiate and train a LinearLearner

optimized_linear_learner = LinearLearner(role=role,
                                         instance_count=1,
                                         instance_type='ml.m4.xlarge',
                                         sagemaker_session=sagemaker_session,
                                         output_path=output_path,
                                         predictor_type='binary_classifier',
                                         epochs=20,
                                         binary_classifier_model_selection_criteria='precision_at_target_recall',
                                         target_recall=0.95,
                                         positive_example_weight_mult='balanced')

# binary_classifier_model_selection_criteria='recall_at_target_precision'
# target_precision=0.90

CPU times: user 1.91 ms, sys: 0 ns, total: 1.91 ms
Wall time: 1.93 ms


In [34]:
%%time 
# deploy and evaluate a predictor
optimized_linear_learner.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-07-15 22:20:15 Starting - Starting the training job...
2021-07-15 22:20:38 Starting - Launching requested ML instancesProfilerReport-1626387615: InProgress
......
2021-07-15 22:21:38 Starting - Preparing the instances for training...........................
2021-07-15 22:26:17 Starting - Launched instance was unhealthy, replacing it!......
2021-07-15 22:26:59 Starting - Preparing the instances for training.........
2021-07-15 22:28:40 Downloading - Downloading input data...
2021-07-15 22:29:03 Training - Downloading the training image...
2021-07-15 22:29:40 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[07/15/2021 22:29:35 INFO 140403100632896] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_mo

[2021-07-15 22:30:00.419] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 8206, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626388200.419489, "EndTime": 1626388200.4195821, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.28058536192160755, "count": 1, "min": 0.28058536192160755, "max": 0.28058536192160755}}}

#metrics {"StartTime": 1626388200.419693, "EndTime": 1626388200.4197204, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.26406076670891077, "count": 1, "min": 0.26406076670891077, "max": 0.26406076670891077}}}

#metrics {"StartTime": 1626388200.419777, "EndTime": 1626388200.4197955, "Dimensions": {"Algorithm

[2021-07-15 22:30:09.328] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 8890, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626388209.3281858, "EndTime": 1626388209.3282764, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2675552611231205, "count": 1, "min": 0.2675552611231205, "max": 0.2675552611231205}}}

#metrics {"StartTime": 1626388209.328367, "EndTime": 1626388209.328393, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.255248536421426, "count": 1, "min": 0.255248536421426, "max": 0.255248536421426}}}

#metrics {"StartTime": 1626388209.3284485, "EndTime": 1626388209.3284662, "Dimensions": {"Algorithm": "Line

[2021-07-15 22:30:17.019] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 7672, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626388217.0191078, "EndTime": 1626388217.019217, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2603724578397358, "count": 1, "min": 0.2603724578397358, "max": 0.2603724578397358}}}

#metrics {"StartTime": 1626388217.0193164, "EndTime": 1626388217.0193405, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.250463905813706, "count": 1, "min": 0.250463905813706, "max": 0.250463905813706}}}

#metrics {"StartTime": 1626388217.0193994, "EndTime": 1626388217.0194178, "Dimensions": {"Algorithm": "Li

...

[2021-07-15 22:31:51.988] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 35, "duration": 7993, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626388311.9885767, "EndTime": 1626388311.9887152, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 16, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.24311471753144384, "count": 1, "min": 0.24311471753144384, "max": 0.24311471753144384}}}

#metrics {"StartTime": 1626388311.988836, "EndTime": 1626388311.9888604, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 16, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.23880562583885, "count": 1, "min": 0.23880562583885, "max": 0.23880562583885}}}

#metrics {"StartTime": 1626388311.9889271, "EndTime": 1626388311.988947, "Dimensions": {"Algorithm": "L

[2021-07-15 22:32:00.160] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 37, "duration": 8155, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626388320.1607409, "EndTime": 1626388320.1608636, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 17, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.24287452291364048, "count": 1, "min": 0.24287452291364048, "max": 0.24287452291364048}}}

#metrics {"StartTime": 1626388320.161054, "EndTime": 1626388320.16108, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 17, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.23850908377182545, "count": 1, "min": 0.23850908377182545, "max": 0.23850908377182545}}}

#metrics {"StartTime": 1626388320.1612136, "EndTime": 1626388320.1612315, "Dimensions": {"Algori


2021-07-15 22:32:21 Uploading - Uploading generated training model[2021-07-15 22:32:08.921] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 39, "duration": 8744, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626388328.92147, "EndTime": 1626388328.9215736, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 18, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2426776494931935, "count": 1, "min": 0.2426776494931935, "max": 0.2426776494931935}}}

#metrics {"StartTime": 1626388328.9216871, "EndTime": 1626388328.9217112, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 18, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.23823233894846546, "count": 1, "min": 0.23823233894846546, "max": 0.23823233894846546}}}

#metrics {"StartTime": 16263883

[2021-07-15 22:32:16.565] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 41, "duration": 7627, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626388336.5652633, "EndTime": 1626388336.565371, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 19, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2425170233261645, "count": 1, "min": 0.2425170233261645, "max": 0.2425170233261645}}}

#metrics {"StartTime": 1626388336.5654774, "EndTime": 1626388336.5655031, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 19, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.23797676945211899, "count": 1, "min": 0.23797676945211899, "max": 0.23797676945211899}}}

#metrics {"StartTime": 1626388336.5655744, "EndTime": 1626388336.5655947, "Dimensions": {"Algorit


2021-07-15 22:32:41 Completed - Training job completed
ProfilerReport-1626387615: NoIssuesFound
Training seconds: 235
Billable seconds: 235
CPU times: user 1.68 s, sys: 91.6 ms, total: 1.77 s
Wall time: 12min 46s


In [35]:
optimized_predictor = optimized_linear_learner.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


---------------!

In [38]:
print('Metrics for optimized, LinearLearner.\n')

# get metrics for balanced predictor
metrics = evaluate(optimized_predictor, 
                   test_features.values.astype('float32'), 
                   test_labels, 
                   verbose=True)

Metrics for optimized, LinearLearner.

prediction (col)    0.0   1.0
actual (row)                 
0                 76669  8635
1                     4   135

Recall:     0.971
Precision:  0.015
Accuracy:   0.899



In [39]:
optimized_predictor.delete_endpoint()

## Final Cleanup!

* Double check that you have deleted all your endpoints.
* I'd also suggest manually deleting your S3 bucket, models, and endpoint configurations directly from your AWS console.

You can find thorough cleanup instructions, [in the documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html).

---
# Conclusion

In this notebook, you saw how to train and deploy a LinearLearner in SageMaker. This model is well-suited for a binary classification task that involves specific design decisions and managing class imbalance in the training set.

Following the steps of a machine learning workflow, you loaded in some credit card transaction data, explored that data and prepared it for model training. Then trained, deployed, and evaluated several models, according to different design considerations!